In [1]:
%matplotlib inline
from pathlib import *
import pandas as pd
import numpy as np
import math
import statsmodels as sm
from scipy import stats as ss
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))
pd.set_option('display.width', 999999)
pd.options.display.max_colwidth = 250
pd.options.display.max_rows = 999
from pylabs.projects.bbc.pairing import dwipairing
from pylabs.utils.paths import getnetworkdataroot
fs = Path(getnetworkdataroot())
#get data from test file
LCSP_fname = fs / 'bbc' / 'LeftPostIntCap.csv'
lcsp_data = pd.read_csv(LCSP_fname, index_col=0, header=None)
lcsp_data.reset_index(level=0, inplace=True)

In [2]:
#makes subj id fm 1st 10 chars of vtk_file_name
lcsp_data['Subjid'] = lcsp_data.iloc[:, 0].str[0:10]
lcsp_data.head(4)

,0,1,2,3,4,5,6,7,8,9,10,11,Subjid
0,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_LeftPostIntCap-35_channel_reg2vbmtempl_FA_AD_RD_MD.nii,fa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub-bbc101
1,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_LeftPostIntCap-35_channel_reg2vbmtempl_FA_AD_RD_MD.nii,ad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub-bbc101
2,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_LeftPostIntCap-35_channel_reg2vbmtempl_FA_AD_RD_MD.nii,rd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub-bbc101
3,sub-bbc101_ses-2_dti_15dir_b1000_1_eddy_corrected_repol_std2_wls_fsl_tensor_medfilt_mori_LeftPostIntCap-35_channel_reg2vbmtempl_FA_AD_RD_MD.nii,md,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sub-bbc101


In [3]:
lcsp_data.rename(columns = {lcsp_data.columns[0]: 'fname', lcsp_data.columns[1]: 'modality', lcsp_data.columns[2]: 'seg1', lcsp_data.columns[3]: 'seg2', lcsp_data.columns[4]: 'seg3', lcsp_data.columns[5]: 'seg4', lcsp_data.columns[6]: 'seg5', lcsp_data.columns[7]: 'seg6', lcsp_data.columns[8]: 'seg7', lcsp_data.columns[9]: 'seg8', lcsp_data.columns[10]: 'seg9', lcsp_data.columns[11]: 'seg10'}, inplace = True)
lcsp_data.drop('fname', axis=1, inplace=True)

In [4]:
lcsp_data.set_index(['Subjid', 'modality'], inplace=True)
lcsp_data.head(7)

seg1      seg2      seg3      seg4      seg5      seg6      seg7      seg8      seg9     seg10
Subjid     modality                                                                                                    
sub-bbc101 fa             NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN
           ad             NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN
           rd             NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN
           md             NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN
sub-bbc105 fa        0.055000  0.143661  0.133353  0.091900  0.212808  0.312858  0.230360  0.185330  0.179046  0.089200
           ad        0.000293  0.000411  0.000416  0.000441  0.000329  0.000230  0.000284  0.000297  0.000310  0.000343
           rd        0.000283  0.000251  0.000325  0.000482  0.000437  0.000444  0.000478  0.000382  0.000383  0.000436

In [5]:
lcsp_data = lcsp_data.unstack(0)
lcsp_data.head(7)

seg1                                                                                                       ...          seg10                                                                                                   
Subjid   sub-bbc101 sub-bbc105 sub-bbc106 sub-bbc108 sub-bbc113 sub-bbc116 sub-bbc118 sub-bbc119 sub-bbc120 sub-bbc202    ...     sub-bbc120 sub-bbc202 sub-bbc208 sub-bbc209 sub-bbc211 sub-bbc231 sub-bbc241 sub-bbc243 sub-bbc249 sub-bbc253
modality                                                                                                                  ...                                                                                                                  
ad              NaN   0.000293   0.000299   0.000361        NaN   0.000317   0.000330   0.000209   0.000160   0.000219    ...       0.000338   0.000275   0.000371   0.000330   0.000212   0.000317   0.000337   0.000275   0.000377   0.000352
fa              NaN   0.055000   0.091100   0.066700        NaN   0.060300   0.033200   0.103622   0.046200   0.036400    ...       0.052800   0.053500   0.066100   0.072900   0.039100   0.053000   0.056900   0.046700   0.090100   0.073700
md              NaN   0.000286   0.000237   0.000393        NaN   0.000324   0.000340   0.000261   0.000146   0.000240    ...       0.000349   0.000312   0.000388   0.000373   0.000243   0.000330   0.000380   0.000291   0.000391   0.000388
rd              NaN   0.000283   0.000206   0.000409        NaN   0.000327   0.000345   0.000287   0.000139   0.000250    ...       0.000354   0.000330   0.000396   0.000395   0.000258   0.000336   0.000401   0.000299   0.000398   0.000406

[4 rows x 180 columns]

In [6]:
lcsp_data = lcsp_data.stack(0)
lcsp_data.head(7)

Subjid         sub-bbc101  sub-bbc105  sub-bbc106  sub-bbc108  sub-bbc113  sub-bbc116  sub-bbc118  sub-bbc119  sub-bbc120  sub-bbc202  sub-bbc208  sub-bbc209  sub-bbc211  sub-bbc231  sub-bbc241  sub-bbc243  sub-bbc249  sub-bbc253
modality                                                                                                                                                                                                                             
ad       seg1         NaN    0.000293    0.000299    0.000361         NaN    0.000317    0.000330    0.000209    0.000160    0.000219    0.000241    0.000195    0.000178    0.000362    0.000263    0.000158    0.000228    0.000126
         seg2         NaN    0.000411    0.000255    0.000429         NaN    0.000240    0.000327    0.000322    0.000228    0.000366    0.000300    0.000296    0.000304    0.000519    0.000383    0.000298    0.000270    0.000173
         seg3         NaN    0.000416    0.000397    0.000278         NaN    0.000380    0.000374    0.000338    0.000447    0.000310    0.000271    0.000301    0.000194    0.000343    0.000383    0.000503    0.000222    0.000299
         seg4         NaN    0.000441    0.000232    0.000274         NaN    0.000335    0.000297    0.000279    0.000310    0.000240    0.000441    0.000409    0.000132    0.000298    0.000261    0.000290    0.000352    0.000323
         seg5         NaN    0.000329    0.000208    0.000269         NaN    0.000211    0.000202    0.000225    0.000262    0.000218    0.000322    0.000290    0.000110    0.000296    0.000221    0.000273    0.000295    0.000267
         seg6         NaN    0.000230    0.000195    0.000181         NaN    0.000198    0.000264    0.000203    0.000223    0.000229    0.000244    0.000222    0.000142    0.000223    0.000213    0.000212    0.000255    0.000221
         seg7         NaN    0.000284    0.000275    0.000287         NaN    0.000194    0.000412    0.000278    0.000269    0.000306    0.000273    0.000232    0.000188    0.000316    0.000267    0.000270    0.000258    0.000325

In [7]:
lcsp_data = lcsp_data.unstack(0)
lcsp_data.head(7)

Subjid   sub-bbc101             sub-bbc105                               sub-bbc106              ...    sub-bbc243           sub-bbc249                               sub-bbc253                              
modality         ad  fa  md  rd         ad        fa        md        rd         ad        fa    ...            md        rd         ad        fa        md        rd         ad        fa        md        rd
seg1            NaN NaN NaN NaN   0.000293  0.055000  0.000286  0.000283   0.000299  0.091100    ...      0.000140  0.000131   0.000228  0.061800  0.000224  0.000221   0.000126  0.065300  0.000167  0.000187
seg2            NaN NaN NaN NaN   0.000411  0.143661  0.000304  0.000251   0.000255  0.141154    ...      0.000289  0.000284   0.000270  0.080600  0.000278  0.000282   0.000173  0.120251  0.000244  0.000280
seg3            NaN NaN NaN NaN   0.000416  0.133353  0.000355  0.000325   0.000397  0.161847    ...      0.000511  0.000514   0.000222  0.118551  0.000272  0.000297   0.000299  0.087800  0.000348  0.000372
seg4            NaN NaN NaN NaN   0.000441  0.091900  0.000469  0.000482   0.000232  0.268484    ...      0.000346  0.000374   0.000352  0.092400  0.000392  0.000412   0.000323  0.169118  0.000417  0.000464
seg5            NaN NaN NaN NaN   0.000329  0.212808  0.000401  0.000437   0.000208  0.326231    ...      0.000337  0.000370   0.000295  0.134617  0.000322  0.000335   0.000267  0.228059  0.000351  0.000393
seg6            NaN NaN NaN NaN   0.000230  0.312858  0.000372  0.000444   0.000195  0.299917    ...      0.000342  0.000407   0.000255  0.260352  0.000346  0.000391   0.000221  0.338427  0.000391  0.000475
seg7            NaN NaN NaN NaN   0.000284  0.230360  0.000413  0.000478   0.000275  0.182723    ...      0.000366  0.000414   0.000258  0.241166  0.000390  0.000456   0.000325  0.240394  0.000438  0.000495

[7 rows x 72 columns]

In [16]:
lcsp_data['211-105'] = lcsp_data['sub-bbc211']['fa'] - lcsp_data['sub-bbc105']['fa']
lcsp_data['211-105']

seg1    -0.026200
seg2    -0.063261
seg3    -0.005131
seg4     0.111412
seg5    -0.043599
seg6    -0.167487
seg7    -0.160060
seg8    -0.097730
seg9    -0.119846
seg10   -0.050100
Name: 211-105, dtype: float64

In [13]:
lcsp_data.head(7)

Subjid   sub-bbc101             sub-bbc105                               sub-bbc106              ...    sub-bbc243 sub-bbc249                               sub-bbc253                                 211-105
modality         ad  fa  md  rd         ad        fa        md        rd         ad        fa    ...            rd         ad        fa        md        rd         ad        fa        md        rd          
seg1            NaN NaN NaN NaN   0.000293  0.055000  0.000286  0.000283   0.000299  0.091100    ...      0.000131   0.000228  0.061800  0.000224  0.000221   0.000126  0.065300  0.000167  0.000187 -0.026200
seg2            NaN NaN NaN NaN   0.000411  0.143661  0.000304  0.000251   0.000255  0.141154    ...      0.000284   0.000270  0.080600  0.000278  0.000282   0.000173  0.120251  0.000244  0.000280 -0.063261
seg3            NaN NaN NaN NaN   0.000416  0.133353  0.000355  0.000325   0.000397  0.161847    ...      0.000514   0.000222  0.118551  0.000272  0.000297   0.000299  0.087800  0.000348  0.000372 -0.005131
seg4            NaN NaN NaN NaN   0.000441  0.091900  0.000469  0.000482   0.000232  0.268484    ...      0.000374   0.000352  0.092400  0.000392  0.000412   0.000323  0.169118  0.000417  0.000464  0.111412
seg5            NaN NaN NaN NaN   0.000329  0.212808  0.000401  0.000437   0.000208  0.326231    ...      0.000370   0.000295  0.134617  0.000322  0.000335   0.000267  0.228059  0.000351  0.000393 -0.043599
seg6            NaN NaN NaN NaN   0.000230  0.312858  0.000372  0.000444   0.000195  0.299917    ...      0.000407   0.000255  0.260352  0.000346  0.000391   0.000221  0.338427  0.000391  0.000475 -0.167487
seg7            NaN NaN NaN NaN   0.000284  0.230360  0.000413  0.000478   0.000275  0.182723    ...      0.000414   0.000258  0.241166  0.000390  0.000456   0.000325  0.240394  0.000438  0.000495 -0.160060

[7 rows x 73 columns]

In [ ]:
lcsp_data.set_index(['Subjid', 'modality'])

In [ ]:
list(result.columns.levels[0])

In [ ]:
lcsp_data.unstack(level=0)
lcsp_data.unstack(level=0)
lcsp_data.T
lcsp_data

In [12]:
lcsp_data.T
lcsp_data

,0,1,2,3,4,5,6,7,8,9,...,62,63,64,65,66,67,68,69,70,71
modality,fa,ad,rd,md,fa,ad,rd,md,fa,ad,...,rd,md,fa,ad,rd,md,fa,ad,rd,md
seg1,NaN,NaN,NaN,NaN,0.055,0.000293,0.000283,0.000286,0.0911,0.000299,...,0.000131,0.00014,0.0618,0.000228,0.000221,0.000224,0.0653,0.000126,0.000187,0.000167
seg2,NaN,NaN,NaN,NaN,0.143661,0.000411,0.000251,0.000304,0.141154,0.000255,...,0.000284,0.000289,0.0806,0.00027,0.000282,0.000278,0.120251,0.000173,0.00028,0.000244
seg3,NaN,NaN,NaN,NaN,0.133353,0.000416,0.000325,0.000355,0.161847,0.000397,...,0.000514,0.000511,0.118551,0.000222,0.000297,0.000272,0.0878,0.000299,0.000372,0.000348
seg4,NaN,NaN,NaN,NaN,0.0919,0.000441,0.000482,0.000469,0.268484,0.000232,...,0.000374,0.000346,0.0924,0.000352,0.000412,0.000392,0.169118,0.000323,0.000464,0.000417
seg5,NaN,NaN,NaN,NaN,0.212808,0.000329,0.000437,0.000401,0.326231,0.000208,...,0.00037,0.000337,0.134617,0.000295,0.000335,0.000322,0.228059,0.000267,0.000393,0.000351
seg6,NaN,NaN,NaN,NaN,0.312858,0.00023,0.000444,0.000372,0.299917,0.000195,...,0.000407,0.000342,0.260352,0.000255,0.000391,0.000346,0.338427,0.000221,0.000475,0.000391
seg7,NaN,NaN,NaN,NaN,0.23036,0.000284,0.000478,0.000413,0.182723,0.000275,...,0.000414,0.000366,0.241166,0.000258,0.000456,0.00039,0.240394,0.000325,0.000495,0.000438
seg8,NaN,NaN,NaN,NaN,0.18533,0.000297,0.000382,0.000353,0.166965,0.000353,...,0.000322,0.000332,0.214399,0.000394,0.000362,0.000372,0.211872,0.000404,0.000455,0.000438
seg9,NaN,NaN,NaN,NaN,0.179046,0.00031,0.000383,0.000358,0.0989,0.000308,...,0.000379,0.000376,0.199903,0.000331,0.000417,0.000388,0.160783,0.000344,0.000454,0.000417
